<a href="https://colab.research.google.com/github/Muniba08/Bike-Sharing-Multiple-Linear-Regression-Modelling/blob/main/Muniba_BlackCoffer_Test_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing library
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
#from google.colab import files
import nltk
import string
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
#importing input file
new_df=pd.read_csv(r"/content/drive/MyDrive/Black coffer/20211030 Test Assignment-20240713T115804Z-001/20211030 Test Assignment/Input.csv")[['URL_ID','URL']]

In [4]:
new_df.head()

,URL_ID,URL
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...
3,bctech2014,https://insights.blackcoffer.com/effective-man...
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...


In [5]:
df=new_df.drop('URL_ID',axis=1)

In [6]:
url_list = [url for url in df['URL']]

In [7]:
output=pd.DataFrame( columns=['url_id','url','positive_score','negative_score','Polarity_Score','subjectiivity_score','avg_senetence_length','Percentage_of_Complex_words','Fog_Index','avg_no_of_words_per_sentence','complex_Word_Count','word_count','syllable_count','personal_pronouns','avg_word_length'])


**Data Extraction**

In [22]:
# Function to ensure the URL has a scheme (http or https)
def ensure_url_scheme(url):
    if pd.isna(url):
        return None
    if not url.startswith(('http://', 'https://')):
        return 'http://' + url
    return url

url_id=1
for i in range(0,len(df)):
   number=i
   j=df.iloc[i].values
   url = ensure_url_scheme(j[0])

   if url is None:
    continue

   try:
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}#giving user access
    page=requests.get(j[0],headers=headers)#loading text in url

    soup=BeautifulSoup(page.content,'html.parser')#parsing url text
    content=soup.findAll(attrs={'class':'td-post-content'})#extracting only text part
    if not content:
      continue
    content=content[0].text.replace('\xa0',"  ").replace('\n',"  ")#replace end line symbol with space
    title=soup.findAll(attrs={'class':'entry-title'})#extracting title of website
    if not title:
      continue
    title=title[16].text.replace('\n',"  ").replace('/',"")
    text=title+ '.' +content#merging title and content text
    text=np.array(text)#converting to array form
    text.reshape(1,-1)#changing shape to 1d
    df1=pd.Series(text)#creating series data frame
    b=str(url_id)+"."+'txt'#name of the text file
    with open(b, 'a') as f:#creating text file
      df1.to_csv(f, lineterminator=',', index=False, header=False)
    #files.download(b)#downloading text file
    url_id+=1
     #importing each extracted files
    text=pd.read_csv("/content/"+str(b),header=None)

    #information of data frame
    text.info()

    #removing extra created column
    text.drop(1,axis=1,inplace=True)

    #converting type
    text=text.astype(str)

    #converting text to sentence
    import re
    a=text[0].str.split('([\.]\s)',expand=False)#splitting text on '.'
    b=a.explode()#converting to rows
    b=pd.DataFrame(b)#creating data frame
    b.columns=['abc']

    #removing . char from each rows
    def abcd(x):
        nopunc =[char for char in x if char != '.']
        return ''.join(nopunc)
    b['abc']=b['abc'].apply(abcd)

    #replacing emty space with null values
    c=b.replace('',np.nan,regex=True)
    c=c.mask(c==" ")
    c=c.dropna()
    c.reset_index(drop=True,inplace=True)

    c

    #importing nltk library and stopwords
    import nltk
    import string

    punc=[punc for punc in string.punctuation]

    punc

    #importing stop words files that are provided
    StopWords_Auditor=pd.read_csv("/content/drive/MyDrive/Black coffer/20211030 Test Assignment-20240713T115804Z-001/20211030 Test Assignment/StopWords_Auditor.txt",header=None)
    StopWords_Currencies=pd.read_csv("/content/drive/MyDrive/Black coffer/20211030 Test Assignment-20240713T115804Z-001/20211030 Test Assignment/StopWords_Currencies.txt",header=None,encoding="ISO-8859-1",sep='/n' )
    StopWords_DatesandNumbers=pd.read_csv("/content/drive/MyDrive/Black coffer/20211030 Test Assignment-20240713T115804Z-001/20211030 Test Assignment/StopWords_DatesandNumbers.txt",header=None)
    StopWords_Generic=pd.read_csv("/content/drive/MyDrive/Black coffer/20211030 Test Assignment-20240713T115804Z-001/20211030 Test Assignment/StopWords_Generic.txt",header=None)
    StopWords_GenericLong=pd.read_csv("/content/drive/MyDrive/Black coffer/20211030 Test Assignment-20240713T115804Z-001/20211030 Test Assignment/StopWords_GenericLong.txt",header=None)
    StopWords_Geographic=pd.read_csv("/content/drive/MyDrive/Black coffer/20211030 Test Assignment-20240713T115804Z-001/20211030 Test Assignment/StopWords_Geographic.txt",header=None)
    StopWords_Names=pd.read_csv("/content/drive/MyDrive/Black coffer/20211030 Test Assignment-20240713T115804Z-001/20211030 Test Assignment/StopWords_Names.txt",header=None)

    #creating func for removing stop words
    def text_process(text):
        nopunc =[char for char in text if char not in punc or char not in [':',',','(',')','’','?']]
        nopunc=''.join(nopunc)
        txt=' '.join([word for word in nopunc.split() if word.lower() not in StopWords_Auditor])
        txt1=' '.join([word for word in txt.split() if word.lower() not in StopWords_Currencies])
        txt2=' '.join([word for word in txt1.split() if word.lower() not in StopWords_DatesandNumbers])
        txt3=' '.join([word for word in txt2.split() if word.lower() not in StopWords_Generic])
        txt4=' '.join([word for word in txt3.split() if word.lower() not in StopWords_GenericLong])
        txt5=' '.join([word for word in txt4.split() if word.lower() not in StopWords_Geographic])
        return ' '.join([word for word in txt5.split() if word.lower() not in StopWords_Names])

    #applying func for each row
    c['abc']=c['abc'].apply(text_process)

    c

    #importing master Dictionary
    positive=pd.read_csv("/content/drive/MyDrive/Black coffer/20211030 Test Assignment-20240713T115804Z-001/20211030 Test Assignment/MasterDictionary/positive-words.txt",header=None)
    negative=pd.read_csv("/content/drive/MyDrive/Black coffer/20211030 Test Assignment-20240713T115804Z-001/20211030 Test Assignment/MasterDictionary/negative-words.txt",header=None,encoding="ISO-8859-1",sep='/n' )

    positive.columns=['abc']
    negative.columns=['abc']
    positive['abc']=positive['abc'].astype(str)
    negative['abc']=negative['abc'].astype(str)

    #positive and negative dictionary without stopwords
    positive['abc']=positive['abc'].apply(text_process)
    negative['abc']=negative['abc'].apply(text_process)

    #positive list
    length=positive.shape[0]
    post=[]
    for i in range(0,length):
      nopunc =[char for char in positive.iloc[i] if char not in string.punctuation or char != '+']
      nopunc=''.join(nopunc)

      post.append(nopunc)

    #negative list
    length=negative.shape[0]
    neg=[]
    for i in range(0,length):
      nopunc =[char for char in negative.iloc[i] if char not in string.punctuation or char != '+']
      nopunc=''.join(nopunc)
      neg.append(nopunc)

    #importing tokenize library
    from nltk.tokenize import word_tokenize

    txt_list=[]
    length=c.shape[0]
    for i in range(0,length):
      txt=' '.join([word for word in c.iloc[i]])
      txt_list.append(txt)

    #tokenization of text
    tokenize_text=[]
    for i in txt_list:

      tokenize_text+=(word_tokenize(i))

    print(tokenize_text)

    len(tokenize_text)

    """### 1) POSITIVE SCORE"""

    positive_score=0
    for i in tokenize_text:
      if(i.lower() in post):
        positive_score+=1
    print('postive score=', positive_score)

    """### 2) NEGATIVE SCORE"""

    negative_score=0
    for i in tokenize_text:
      if(i.lower() in neg):
        negative_score+=1
    print('negative score=', negative_score)

    """### 3) POLARITY SCORE"""

    #Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
    Polarity_Score=(positive_score-negative_score)/((positive_score+negative_score)+0.000001)
    print('polarity_score=', Polarity_Score)

    """### 4) SUBJECTIVITY SCORE"""

    #Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
    subjectiivity_score=(positive_score-negative_score)/((len(tokenize_text))+ 0.000001)
    print('subjectivity_score',subjectiivity_score)

    """### 5) AVG SENTENCE LENGTH"""

    length=c.shape[0]
    avg_length=[]
    for i in range(0,length):
      avg_length.append(len(c['abc'].iloc[i]))
    avg_senetence_length=sum(avg_length)/len(avg_length)
    print('avg sentence length=', avg_senetence_length)

    """### 6) PERCENTAGE OF COMPLEX WORDS"""

    vowels=['a','e','i','o','u']
    import re
    count=0
    complex_Word_Count=0
    for i in tokenize_text:
      x=re.compile('[es|ed]$')
      if x.match(i.lower()):
        count+=0
      else:
        for j in i:
          if(j.lower() in vowels ):
            count+=1
      if(count>2):
        complex_Word_Count+=1
      count=0

    Percentage_of_Complex_words=complex_Word_Count/len(tokenize_text)
    print('percentag of complex words= ',Percentage_of_Complex_words)

    """### 7) FOG INDEX"""

    #Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
    Fog_Index = 0.4 * (avg_senetence_length + Percentage_of_Complex_words)
    print('fog index= ',Fog_Index )

    """### 8) AVG NUMBER OF WORDS PER SENTENCE"""

    length=c.shape[0]
    avg_length=[]
    for i in range(0,length):
      a=[word.split( ) for word in c.iloc[i]]
      avg_length.append(len(a[0]))
      a=0
    #avg
    avg_no_of_words_per_sentence=sum(avg_length)/length
    print("avg no of words per sentence= ",avg_no_of_words_per_sentence)

    """### 9) COMPLEX WORD COUNT"""

    vowels=['a','e','i','o','u']
    import re
    count=0
    complex_Word_Count=0
    for i in tokenize_text:
      x=re.compile('[es|ed]$')
      if x.match(i.lower()):
        count+=0
      else:
        for j in i:
          if(j.lower() in vowels ):
            count+=1
      if(count>2):
        complex_Word_Count+=1
      count=0
    print('complex words count=',  complex_Word_Count)

    """### 10) WORD COUNT"""

    word_count=len(tokenize_text)
    print('word count= ', word_count)

    """### 11) SYLLABLE PER WORD"""

    vowels=['a','e','i','o','u']
    import re
    count=0
    for i in tokenize_text:
      x=re.compile('[es|ed]$')
      if x.match(i.lower()):
        count+=0
      else:
        for j in i:
          if(j.lower() in vowels ):
            count+=1
    syllable_count=count
    print('syllable_per_word= ',syllable_count)

    """### 12) PERSONAL PRONOUNS"""

    pronouns=['i','we','my','ours','us' ]
    import re
    count=0
    for i in tokenize_text:
      if i.lower() in pronouns:
        count+=1
    personal_pronouns=count
    print('personal pronouns= ',personal_pronouns )

    """### 13) AVG WORD LENGTH"""

    count=0
    for i in tokenize_text:
      for j in i:
        count+=1
    avg_word_length=count/len(tokenize_text)
    print('avg word= ', avg_word_length)

    data={'url_id':new_df['URL_ID'][number],'url':new_df['URL'][number],'positive_score':positive_score,'negative_score':negative_score,'Polarity_Score':Polarity_Score,'subjectivity_score':subjectivity_score,'avg_sentence_length':avg_sentence_length,'Percentage_of_Complex_words':Percentage_of_Complex_words,'Fog_Index':Fog_Index,'avg_no_of_words_per_sentence':avg_no_of_words_per_sentence,'complex_Word_Count':complex_Word_Count,'word_count':word_count,'syllable_count':syllable_count,'personal_pronouns':personal_pronouns,'avg_word_length':avg_word_length}


    output=output.append(data,ignore_index=True)
    output


   except Exception as e:
     url_id+=1
     print(e)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1 non-null      object 
 1   1       1 non-null      object 
 2   2       1 non-null      object 
 3   3       0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 160.0+ bytes
[Errno 2] No such file or directory: '/content/drive/MyDrive/Black coffer/20211030 Test Assignment-20240713T115804Z-001/20211030 Test Assignment/StopWords_Auditor.txt'
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1 non-null      object 
 1   1       1 non-null      object 
 2   2       1 non-null      object 
 3   3       0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 160.0+ bytes
[Errno 2] No such file or directory: '/content/drive/MyDrive/Black coffer/

In [ ]:
output

,url_id,url,positive_score,negative_score,Polarity_Score,subjectiivity_score,avg_senetence_length,Percentage_of_Complex_words,Fog_Index,avg_no_of_words_per_sentence,complex_Word_Count,word_count,syllable_count,personal_pronouns,avg_word_length
0,37,https://insights.blackcoffer.com/ai-in-healthc...,79,34,0.398230,0.024712,155.155844,0.348160,62.201602,23.324675,634,1821,3922,1,5.616694
1,37,https://insights.blackcoffer.com/ai-in-healthc...,79,34,0.398230,0.024712,155.155844,0.348160,62.201602,23.324675,634,1821,3922,1,5.616694
2,38,https://insights.blackcoffer.com/what-if-the-c...,71,38,0.302752,0.022634,118.071429,0.233196,47.321850,20.242857,340,1458,2667,6,4.744170
3,39,https://insights.blackcoffer.com/what-jobs-wil...,74,38,0.321429,0.020713,126.686047,0.330265,50.806524,19.837209,574,1738,3608,3,5.336594
4,40,https://insights.blackcoffer.com/will-machine-...,85,28,0.504425,0.034030,115.253012,0.234627,46.195056,19.951807,393,1675,3126,18,4.771940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,146,https://insights.blackcoffer.com/blockchain-fo...,29,28,0.017544,0.001070,112.230769,0.283422,45.005677,17.673077,265,935,1821,11,5.314439
108,147,https://insights.blackcoffer.com/the-future-of...,47,14,0.540984,0.020535,159.750000,0.282514,64.013006,25.966667,454,1607,3055,18,5.032358
109,148,https://insights.blackcoffer.com/big-data-anal...,32,48,-0.200000,-0.013468,103.735294,0.308081,41.617350,17.088235,366,1188,2305,3,5.016835
110,149,https://insights.blackcoffer.com/business-anal...,39,4,0.813953,0.047490,161.166667,0.351425,64.607237,24.066667,259,737,1568,0,5.621438


In [25]:
output.to_csv('Output.csv')

In [26]:
import os
print(os.getcwd())


/content


In [27]:
from datetime import datetime
current_date = datetime.now().strftime("%Y%m%d")
file_name = f'/content/drive/MyDrive/Blackcoffer output files/Output_###{current_date}.csv'
output.to_csv(file_name, index=False)
